# 복습과제

In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score,roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도:{1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}, AUC: {4:.4f}'.format(accuracy, precision,recall, f1, roc_auc))

# 문제1

1-1




앙상블 학습유형의 하나인 보팅 방법 중 하드보팅과 소프트보팅의 차이점을 설명해 주세요.


* 하드보팅: 최종 예측값을 다수결로 결정하는 방법
* 각 분류기의 결과값 중 가장 많이 나온 클래스를 최종 예측값으로 사용합니다.


* 소프트보팅: 최종 예측값을 평균값을 비교해 결정하는 방법
* 전체 분류기의 결정 확률 평균이 가장 큰 클래스를 최종 예측값으로 사용합니다. 

1-2



부스팅 방식에 대해 설명해 주세요.

* 여러 개의 약한 학습기를 순차적으로 학습 및 예측 후 각 모델의 예측 결과에 따라 가중치를 다르게 부여하며 오류를 개선해 강분류기로 만드는 방식으로, 특히 잘못 분류된 데이터에 집중하며 분류 규칙을 수정해 나갑니다.

* 배깅과 유사한 방식이지만 배깅은 여러 모델을 동시에 학습하는 반면, 부스팅은 동일한 모델을 순차적으로 학습 후 가중 투표를 합니다.

* Adaptive Boosting과 Gradient Boosting 등이 있습니다. 

1-3



XGBOOST 와 LIGHTGBM 이 GBM보다 더 선호되는 이유를 설명해 주세요.

* GBM이 예측 성능이 뛰어나긴 하지만 학습 시간이 너무 오래 걸리기 때문에 XGBoost나 LightGBM 같은 속도를 개선한 모델을 사용합니다.
* 병렬 처리가 안되는 GBM을 개선해 시간을 단축할 수 있으며, 학습 오류가 일정 횟수이상 나아지지 않을 경우 조기중단을 이용해 불필요한 개선을 줄일 수도 있습니다.     
* XGBoost보다는 보통 LightGBM이 더 빠르며 성능 차이도 크지 않지만 적은 데이터셋에서는 과적합을 일으킬 수도 있습니다.(리프 노드를 비대칭 분할하기 때문에)  

# 문제2


2-1



iris데이터를 불러와 변수에 할당해 주세요.

In [3]:
iris = load_iris()

X = iris.data
y = iris.target

df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

2-2


붓꽃데이터의 data와 target을 train_test_split으로 train데이터와 test데이터로 나누어 주세요.



▶ test_size = 0.3, random_state = 156

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=156)

2-3



로지스틱 회귀(LogisticRegression())와 KNN(KNeighborsClassifier())을 모델로 소프트 보팅 기반의 앙상블 모델을 구현해주세요.



▶ n_neighbors = 8

In [5]:
# 개별 모델
lr = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors=8)

# 앙상블 모델 
ensemble =  VotingClassifier(estimators=[('LR', lr), ('KNN', knn)], voting = 'soft')

2-4



소프트 보팅 기반의 앙상블 모델로 구현한 모델의 학습, 예측, 평가 하는 함수를 작성해 주세요.



▶ 평가시 정확도를 소수점 아래 4자리로 나타내 주세요.

In [6]:
# 앙상블 모델 학습/예측/평가
ensemble.fit(X_train, y_train)
pred = ensemble.predict(X_test)
print("Voting 분류기 정확도 {0:.4f}".format(accuracy_score(y_test, pred)))

Voting 분류기 정확도 0.9556


2-5



마찬가지로 로지스틱 회귀와 KNN 모델을 2-2에서 분리한 데이터를 이용하여 학습, 예측, 평가해 주세요.



▶ 평가시 정확도를 소수점 아래 4자리로 나타내 주세요.

In [7]:
# 개별 모델 학습/예측/평가
clfs = [lr, knn]
for clf in clfs:
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    clf_name = clf.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(clf_name, accuracy_score(y_test, pred)))

LogisticRegression 정확도: 0.9111
KNeighborsClassifier 정확도: 0.9556


2-6



GridSearchCV를 이용해 랜덤포레스트의 하이퍼 파라미터를 변경하면서 train 데이터를 학습해 주세요.



▶ params = {'n_estimators':[100], 'max_depth':[6,8,10,12], 'min_samples_leaf':[8,12,18], 'min_samples_split':[8,16,20]}



▶ random_state = 0, cv = 2, n_jobs = -1

In [8]:
params = {'n_estimators':[100], 
          'max_depth':[6,8,10,12], 
          'min_samples_leaf':[8,12,18], 
          'min_samples_split':[8,16,20]}

rf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid = GridSearchCV(rf, param_grid=params, cv=2, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100]})

2-7



최적 하이퍼 파라미터와 그 때의 정확도를 측정해 주세요.



▶ 평가시 정확도를 소수점 아래 4자리로 나타내 주세요.



(힌트: best_params_ , best_score_ 함수 이용)

In [9]:
print('최적 하이퍼파라미터: ', grid.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid.best_score_))

최적 하이퍼파라미터:  {'max_depth': 6, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.9526


# 문제3


* GBM의 경우 처리 시간이 오래 걸리기 때문에 작은 데이터를 선정했으며<br>
데이터가 그렇게 크지 않기 때문에 결과값이 크게 유의미하지 않습니다.


3-1

첨부된 heart 데이터를 변수에 할당해 주세요.

In [10]:
heart = pd.read_csv('C:/Users/USER/BITAmin/data/heart.csv')

• age    나이 (int) <br>
• sex    성별 (1, 0 / int) <br>
• chest pain type (4 values)    가슴 통증 타입 (0 ~ 3 / int)<br>
• resting blood pressure    혈압<br>
• serum cholestoral in mg/dl	혈청 콜레스테롤<br>
• fasting blood sugar > 120 mg/dl	공복 혈당<br>
• resting electrocardiographic results	심전도<br>
• maximum heart rate achieved	최대 심장박동 수<br>
• exercise induced angina	운동 유도 협심증<br>
• oldpeak = ST depression induced by exercise relative to rest	노약 =운동에 의해 유발되는 St 우울증<br>
• the slope of the peak exercise ST segment	ST 세그먼트의 기울기<br>
• number of major vessels (0-3) colored by flourosopy	혈관의수<br>
• thal : 3 = normal; 6 = fixed defect; 7 = reversable defect	A blood disorder called thalassemia <br>
<br>

• **target: 심장병 (0 = no, 1 = yes)**

3-2



get_dummies를 이용하여 카테고리 변수인 cp, thal, slope를 원-핫 인코딩을 해주세요. 인코딩 값을 변수에 넣어주세요.

In [11]:
# heart = pd.get_dummies(heart, columns=['cp', 'thal', 'slope'])

# 카테고리형 변환
heart = heart.astype({'cp':'object',
                      'thal':'object',
                      'slope':'object'})

# 원-핫 인코딩
category = heart[['cp', 'thal', 'slope']]
onehot = pd.get_dummies(category)

3-3



concat함수를 이용하여 원-핫 인코딩한 데이터를 기존의 데이터에 연결해 주신 후 기존 데이터의 cp, thal, slop 값은 삭제해 주세요.

In [12]:
# 데이터 연결
heart = pd.concat([heart, onehot], axis=1)

# 열 삭제
heart = heart.drop(['cp','thal','slope'], axis=1, inplace=False)

3-4



심장병데이터의 target을 변수 X, y로 분리해 주세요.

In [13]:
X = heart.drop('target', axis=1, inplace=False)
y = heart['target']

3-5



target값을 제외한 데이터프레임 X에 StandardScaler 클래스로 표준화를 해주세요.

In [14]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

3-6



분리한 데이터로 train_test_split으로 train데이터와 test데이터로 나누어 주세요.



▶ test_size = 0.2, random_state = 156

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=156)

3-7



GBM 부스팅 방법으로 데이터를 학습, 예측, 평가해 주세요.



▶ random_state = 0


▶ 평가시 정확도를 소수점 아래 4자리로 나타내 주세요.

In [16]:
gbm = GradientBoostingClassifier(random_state=0)
gbm.fit(X_train, y_train)
gbm_pred = gbm.predict(X_test)
gbm_acc = accuracy_score(y_test, gbm_pred)

print('GBM 정확도: {:.4f}'.format(gbm_acc))

GBM 정확도: 0.9016


3-8



사이킷런 래퍼 XGBoost클래스로 데이터를 학습, 예측을 한 후 get_clf_eval함수로 성능평가를 해주세요.



▶ n_estimators = 400, learning_rate = 0.1, max_depth = 3

In [17]:
# get_clf_eval 
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    print('오차행렬 \n', confusion)
    print('정확도: {0:.4f}\n정밀도: {1:.4f}\n재현율: {2:.4f}\nF1: {3:.4f}\nAUC: {4:.4f}'.format(accuracy, precision, recall, f1, auc))

In [18]:
# XGBoost 학습/예측
xgb = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

# 성능 평가
get_clf_eval(y_test, xgb_pred)

오차행렬 
 [[25  5]
 [ 1 30]]
정확도: 0.9016
정밀도: 0.8571
재현율: 0.9677
F1: 0.9091
AUC: 0.9005


3-9



하이퍼 파라미터를 설정하여 사이킷런 래퍼 XGBoost클래스로 데이터를 학습, 예측을 한 후 get_clf_eval함수로 성능평가를 해주세요.



▶ n_estimators = 400, learning_rate = 0.1, max_depth = 3



▶ early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = True

In [19]:
# 하이퍼파라미터 설정 후 XGBoost 학습/예측
xgb2 = XGBClassifier(n_estimators = 400, learning_rate = 0.1 , max_depth = 3)
evals = [(X_test, y_test)] 
xgb2.fit(X_train, y_train, early_stopping_rounds = 100, 
         eval_metric="logloss", eval_set = evals, verbose=True) 
xgb2_pred = xgb2.predict(X_test)

# 성능 평가
get_clf_eval(y_test, xgb2_pred)

[0]	validation_0-logloss:0.646055
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.607182
[2]	validation_0-logloss:0.570893
[3]	validation_0-logloss:0.543177
[4]	validation_0-logloss:0.520078
[5]	validation_0-logloss:0.495876
[6]	validation_0-logloss:0.476849
[7]	validation_0-logloss:0.459135
[8]	validation_0-logloss:0.443902
[9]	validation_0-logloss:0.431549
[10]	validation_0-logloss:0.418866
[11]	validation_0-logloss:0.408065
[12]	validation_0-logloss:0.398758
[13]	validation_0-logloss:0.391831
[14]	validation_0-logloss:0.38378
[15]	validation_0-logloss:0.377405
[16]	validation_0-logloss:0.371656
[17]	validation_0-logloss:0.367091
[18]	validation_0-logloss:0.364277
[19]	validation_0-logloss:0.362832
[20]	validation_0-logloss:0.35973
[21]	validation_0-logloss:0.356315
[22]	validation_0-logloss:0.350914
[23]	validation_0-logloss:0.348462
[24]	validation_0-logloss:0.341493
[25]	validation_0-logloss:0.33849
[26]	validation_0-logloss:0.3362
[

[230]	validation_0-logloss:0.256739
[231]	validation_0-logloss:0.256835
[232]	validation_0-logloss:0.257295
[233]	validation_0-logloss:0.256574
[234]	validation_0-logloss:0.257601
[235]	validation_0-logloss:0.257276
[236]	validation_0-logloss:0.257589
[237]	validation_0-logloss:0.258053
[238]	validation_0-logloss:0.257286
[239]	validation_0-logloss:0.257477
[240]	validation_0-logloss:0.257389
[241]	validation_0-logloss:0.258206
[242]	validation_0-logloss:0.2588
[243]	validation_0-logloss:0.25916
[244]	validation_0-logloss:0.259032
[245]	validation_0-logloss:0.259685
[246]	validation_0-logloss:0.259867
[247]	validation_0-logloss:0.259547
[248]	validation_0-logloss:0.259988
[249]	validation_0-logloss:0.259083
[250]	validation_0-logloss:0.258263
[251]	validation_0-logloss:0.258362
[252]	validation_0-logloss:0.259286
[253]	validation_0-logloss:0.259664
[254]	validation_0-logloss:0.25935
[255]	validation_0-logloss:0.260269
[256]	validation_0-logloss:0.260712
[257]	validation_0-logloss:0.260

3-10



사이킷런 래퍼 LightGBM클래스로 데이터를 학습, 예측을 한 후 get_clf_eval함수로 성능평가를 해주세요.



▶ n_estimators = 400

In [20]:
# LightGBM 학습/예측
lgbm = LGBMClassifier(n_estimators=400)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_test)

# 성능 평가
get_clf_eval(y_test, lgbm_pred)

오차행렬 
 [[25  5]
 [ 1 30]]
정확도: 0.9016
정밀도: 0.8571
재현율: 0.9677
F1: 0.9091
AUC: 0.9005


3-11



하이퍼 파라미터를 설정하여 사이킷런 래퍼 LightGBM클래스로 데이터를 학습, 예측을 한 후 get_clf_eval함수로 성능평가를 해주세요.


▶ n_estimators = 400



▶ early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = True

In [21]:
# 하이퍼파라미터 설정 후 LightGBM 학습/예측
lgbm2 = LGBMClassifier(n_estimators = 400)
evals = [(X_test, y_test)] 
lgbm2.fit(X_train, y_train, early_stopping_rounds = 100, 
         eval_metric="logloss", eval_set = evals, verbose=True) 
lgbm2_pred = lgbm2.predict(X_test)

# 성능평가
get_clf_eval(y_test, lgbm2_pred)

[1]	valid_0's binary_logloss: 0.644015
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.601156
[3]	valid_0's binary_logloss: 0.562212
[4]	valid_0's binary_logloss: 0.536916
[5]	valid_0's binary_logloss: 0.506428
[6]	valid_0's binary_logloss: 0.486229
[7]	valid_0's binary_logloss: 0.46707
[8]	valid_0's binary_logloss: 0.451735
[9]	valid_0's binary_logloss: 0.437882
[10]	valid_0's binary_logloss: 0.426335
[11]	valid_0's binary_logloss: 0.413759
[12]	valid_0's binary_logloss: 0.404257
[13]	valid_0's binary_logloss: 0.394804
[14]	valid_0's binary_logloss: 0.386711
[15]	valid_0's binary_logloss: 0.379279
[16]	valid_0's binary_logloss: 0.372684
[17]	valid_0's binary_logloss: 0.363005
[18]	valid_0's binary_logloss: 0.354891
[19]	valid_0's binary_logloss: 0.349851
[20]	valid_0's binary_logloss: 0.345137
[21]	valid_0's binary_logloss: 0.339378
[22]	valid_0's binary_logloss: 0.334821
[23]	valid_0's binary_logloss: 0.327429
[24]	valid_0's binary_loglos

3-12


하이퍼 파라미터 early_stopping_rounds 의 의미를 설명해 주시고, early_stopping_rounds를 급격히 줄일 시 예상되는 단점을 설명해 주세요.

* early_stopping_rounds는 조기중단을 설정하는 파라미터로, 지정 횟수 동안 모델이 개선되지 않으면 남은 n_estimators 횟수와 관계없이 학습을 종료합니다. 
* 이는 불필요한 학습을 줄일 수 있기 때문에 학습 수행 시간을 단축할 수 있습니다.
* 단, early_stopping_rounds를 극단적으로 줄이면 모델 개선 여지가 있음에도 학습을 조기종료할 위험이 있기 때문에 적절하게 지정해주어야 합니다.

# 자율문제(데이터가 큰 경우)

자유롭게 푸시는 문제입니다. 피드백은 해드리지 않습니다. 데이터 전처리만 제공하고 배웠던 앙상블을 사용하여 자유롭게 푸시면됩니다.



신용카드 거래 중 사기거래 였던 경우를 감지하는 데이터입니다. 이미 정제된 데이터를 통해 사기인지 아닌지를 구분합니다.



의미없는 'Time'열을 삭제했고, 'Amount'열을 StandardScaler로 데이터를 전처리 했고, <br>
'Class'열과 그외의 열을 구분하여 y와 X에 할당하였습니다.



데이터가 사기가 아닌경우로 치중되어 있기 때문에, SMOTE를 이용하여 언더샘플링을 하였습니다.<br>
언더샘플링을 하지 않으면 시간이 오래 걸립니다! 궁금하신분은 해보시기 바랍니다.



In [22]:
credit = pd.read_csv('C:/Users/USER/BITAmin/data/creditcard.csv')
credit = credit.drop('Time',axis = 1)
scaler = preprocessing.StandardScaler()
credit['std_Amount'] = scaler.fit_transform(credit['Amount'].values.reshape (-1,1))
credit = credit.drop("Amount", axis=1)
X = credit.drop('Class',axis = 1)
y = credit['Class']

In [23]:
credit['Class'].value_counts()
# 데이터가 치중되어 있습니다.

0    284315
1       492
Name: Class, dtype: int64

In [24]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler 

undersample = RandomUnderSampler(sampling_strategy=0.5)
X_under, y_under = undersample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under , test_size = 0.3, random_state=156) 